In [1463]:
import numpy as np
import os
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score

In [1464]:
def load_data(folder):
    x_train = np.load(os.path.join(folder, 'x_train.npy'))
    y_train = np.load(os.path.join(folder, 'y_train.npy'))    
    x_test = np.load(os.path.join(folder, 'x_test.npy'))    
    y_test = np.load(os.path.join(folder, 'y_test.npy'))    
    return x_train, y_train, x_test, y_test

In [1465]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))


class LogisticRegression:
    def __init__(self, dim=2):
        rng = np.random.default_rng(seed=0)
        self.w = rng.normal(size=(dim, 1)) / np.sqrt(dim)
        self.b = np.zeros((1,))
        
    def predict(self, x, probs=False):
        # x - np.array размерности [N, dim]
        #     Массив входных признаков.
        assert x.shape[1] == self.w.shape[0], \
            "Размерность экземпляров данных не соответствует ожидаемой: " + \
            f"ожидалось x.shape[1]={self.w.shape[0]}, но было получено x.shape[1]={x.shape[1]}"
        
        x = x.dot(self.w) + self.b  # logits
        p = sigmoid(x)  # probabilities
        if probs:
            return p
        return np.array(p > 0.5).astype('int32')
        
    def fit(self, x, y, iters=1000, lr=0.01):
        # x - np.array размерности [N, dim]
        #     Массив входных признаков.
        # y - np.array размернсоти [N]
        #     Массив меток (правильных ответов).
        assert len(x) == len(y), \
            "Количество экземпляров в массиве X не равно количеству меток в массиве Y. " + \
            f"Полученные размеры: len(X) = {len(x)}, len(Y) = {len(y)}."
        assert x.shape[1] == self.w.shape[0], \
            "Размерность экземпляров данных не соответствует ожидаемой: " + \
            f"ожидалось x.shape[1]={self.w.shape[0]}, но было получено x.shape[1]={x.shape[1]}"
        # Алгоритм градиентного спуска.
        # Минимизируется бинарная кросс-энтропия.
        y = y.reshape(-1, 1)
        for i in range(iters):
            preds = self.predict(x, probs=True)
            self.w -= lr * np.mean(x.T.dot(preds - y), axis=1, keepdims=True)
            self.b -= lr * np.mean(preds - y, axis=0)
        return self

## 1. Применение логистической регрессии (несбалансированные данные)

### 1.1 Создание и обучение логистической регрессии

In [1466]:
# Указание: производить нормализацию данных не нужно, это часть задания.
x_train, y_train, x_test, y_test = load_data('dataset1')

In [1467]:
# Создайте модель логистической регрессии и обучите её, используя метод fit.

model = LogisticRegression(dim=784)

model.fit(x_train, y_train)


In [1468]:
# Получите предсказания на тестовой выборке и оцените точность модели, 
# используя accuracy_score из пакета SciKit-Learn.

y_pred = model.predict(x_test)

accuracy_reg = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy_reg: .6f}")

Accuracy:  0.904545


### 1.2 Анализ качества модели

In [1469]:
# Допишите класс "глупого классификатора", что всегда предсказывает класс `0`. 

class DummyClassifier:
    def __init__(self):
        print('Hello, brother!')
        
    def predict(self, x):
        # x - numpy массив размерности [N, dim]
        # Должен возвращаться массив N предсказаний
        return np.zeros(x.shape[0], dtype=int)

In [1470]:
# Оцените точность "глупого классификатора", объясните результат.

dummy_classifier = DummyClassifier()
predictions = dummy_classifier.predict(x_test)

accuracy_dum = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy_dum:.6f}")

# accuracy дает не точную картину, так как присутствует дисбаланс классов 
# в данном случае 0 намного больше, чем 1
# отсюда и возникает большой процент в accuracy

Hello, brother!
Accuracy: 0.909091


In [1471]:
# Используйте дополнительные метрики (f1-score, recall, precision) из пакета sklearn для анализа "глупого классификатора".
f1 = f1_score(y_test, predictions)
print(f"F1-score: {f1:.6f}")
    
recall = recall_score(y_test, predictions)
print(f"Recall: {recall:.6f}")
    
precision = precision_score(y_test, predictions, zero_division=0) # добавила параметр, что не было деления на 0
print(f"Precision: {precision:.6f}")

F1-score: 0.000000
Recall: 0.000000
Precision: 0.000000


In [1472]:
# Используя те же метрики, проанализируйте обученную вами модель логистической регрессии.

f1 = f1_score(y_test, y_pred)
print(f"F1-score: {f1:.6f}")
    
recall = recall_score(y_test, y_pred)
print(f"Recall: {recall:.6f}")
    
precision = precision_score(y_test, y_pred)
print(f"Precision: {precision:.6f}")

F1-score: 0.400000
Recall: 0.350000
Precision: 0.466667


In [1473]:
# Объясните результат, описав его комментариями в этой клетке.

# видно, что у "глупого классификатора" метрики равны 0, что говорит о том, 
# что он ни разу не смог предсказать 1 (т.к. он предсказывает только 0)

# для моей модели результаты оказались лучше
# если предсказана 1, то мы уверены в результате на 47%
# из всех предсказаний 1 лишь 35% будет найдено классификатором
# гармоническое среднее будет 40%

### 1.3 Анализ набора данных

In [1474]:
# Посчитайте количество экземпляров данных для каждого класса.
y_test = y_test.astype(int)
        
class_counts = np.bincount(y_test)
for class_label, count in enumerate(class_counts):
    print(f"Class {class_label}: {count}")

Class 0: 200
Class 1: 20


In [1475]:
# Предложите способ улучшения качества модели. Подсказка: добавление дубликатов в данные.
# Указание: не изменяйте тестовую выборку.

class_zero = np.where(y_train == 0)[0]
class_one = np.where(y_train == 1)[0]

num_duplicates = (len(class_zero) - len(class_one))*2

if num_duplicates > 0:
    duplicate = np.random.choice(class_one, size=num_duplicates, replace=True)
    x_train_with_duplicate = np.vstack((x_train, x_train[duplicate]))
    y_train_with_duplicate = np.hstack((y_train, y_train[duplicate]))
else:
    x_train_with_duplicate = x_train
    y_train_with_duplicate = y_train

In [1476]:
# Создайте и обучите модель с использованием предложенных наработок.
model_log = LogisticRegression(dim=784)
model_log.fit(x_train_with_duplicate, y_train_with_duplicate)

In [1477]:
# Оцените качество новой модели, используя метрики из пакета sklearn.metrics. 
# Указание: постарайтесь сбалансировать данные таким образом, чтобы новая модель была ощутимо лучше старой.

predictions_new = model_log.predict(x_test)

accuracy_new = accuracy_score(y_test, predictions_new)
print(f"Accuracy: {accuracy_new:.6f}")
        
f1_new = f1_score(y_test, predictions_new)
print(f"F1-score: {f1_new:.6f}")
        
recall_new = recall_score(y_test, predictions_new)
print(f"Recall: {recall_new:.6f}")
        
precision_new = precision_score(y_test, predictions_new, zero_division=0)
print(f"Precision: {precision_new:.6f}")

# ура!!! модель примерно в полтора-два раза стала лучше 

Accuracy: 0.954545
F1-score: 0.761905
Recall: 0.800000
Precision: 0.727273


## 2. Применение логистической регрессии (нелинейные данные)

In [1478]:
x_train, y_train, x_test, y_test = load_data('dataset2')

In [1479]:
# Создайте и обучите модель но этом наборе данных.
model = LogisticRegression()

model.fit(x_train, y_train)

In [1480]:
# Проанализируйте качество модели.
y_pred = model.predict(x_test)

accuracy_reg = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy_reg: .6f}")

f1 = f1_score(y_test, y_pred)
print(f"F1-score: {f1:.6f}")
    
recall = recall_score(y_test, y_pred)
print(f"Recall: {recall:.6f}")
    
precision = precision_score(y_test, y_pred)
print(f"Precision: {precision:.6f}")

#accuracy и precision небольшие, предсказывает не очень :((, зато recall неплохое

Accuracy:  0.570000
F1-score: 0.619469
Recall: 0.777778
Precision: 0.514706


In [1481]:
# FEATURE ENGINEERING: попробуйте применить на исходных данных разные нелинейные функции (sin, tanh, ...).
# Объедините трансформированные данные с исходными (важно: количество экземпляров в x_train не должно увеличиться).

bigX_train = np.concatenate((x_train, np.sin(x_train), np.cos(x_train)), axis=1)
bigX_test = np.concatenate((x_test, np.sin(x_test), np.cos(x_test)), axis=1)

In [1482]:
# Создайте и обучите модель с использованием наработок.

model = LogisticRegression(dim=6)

model.fit(bigX_train, y_train)

In [1483]:
# Оцените качество новой модели, используя метрики из пакета sklearn.metrics. 
# Указание: постарайтесь добиться точности в 100%!

y_pred = model.predict(bigX_test)

accuracy_reg = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy_reg: .6f}")

f1 = f1_score(y_test, y_pred)
print(f"F1-score: {f1:.6f}")
    
recall = recall_score(y_test, y_pred)
print(f"Recall: {recall:.6f}")
    
precision = precision_score(y_test, y_pred)
print(f"Precision: {precision:.6f}")

Accuracy:  1.000000
F1-score: 1.000000
Recall: 1.000000
Precision: 1.000000


## 3. Доп. задания (любое на выбор, опционально)

### 3.1 'Упрощение' логистической регрессии

Сложность: легко.

In [1484]:
"""
Модифицируйте класс логистической регрессии так, чтобы в нём не использовалась сигмоида.
То есть вывод о предсказанном классе должен делаться на основе значений "до сигмоиды".
Вспомогательная ссылка: https://en.wikipedia.org/wiki/Logit
"""

class MLogisticRegression:
    def __init__(self, dim=2):
        np.random.seed(43)
        self.w = np.random.randn(dim, 1) / np.sqrt(dim)
        self.b = np.zeros((1,))
        
    def predict(self, x, probs=False):
        x = x.dot(self.w) + self.b
        
        return np.array(x > 0).astype('int32')
        
    def fit(self, x, y, iters=1000, lr=0.01):
        y = y.reshape(-1, 1)
        for i in range(iters):
            preds = self.predict(x, probs=True)
            self.w -= lr * np.mean(x.T.dot(preds - y), axis=1, keepdims=True)
            self.b -= lr * np.mean(preds - y, axis=0)

In [1485]:
# Перенесите обученные веса модели из пункта 1.3 в новую модель с модифицированным кодом
x_train, y_train, x_test, y_test = load_data('dataset1')
model_neww = MLogisticRegression()

model_neww.w = model_log.w.copy()
model_neww.b = model_log.b.copy()


In [1486]:
# Убедитесь, что предсказания модели с модифицированными кодом совпадают с предсказаниями
# модели из пункта 1.3
predictions_new = model_neww.predict(x_test)

accuracy_new = accuracy_score(y_test, predictions_new)
print(f"Accuracy: {accuracy_new:.6f}")
        
f1_new = f1_score(y_test, predictions_new)
print(f"F1-score: {f1_new:.6f}")
        
recall_new = recall_score(y_test, predictions_new)
print(f"Recall: {recall_new:.6f}")
        
precision_new = precision_score(y_test, predictions_new, zero_division=0)
print(f"Precision: {precision_new:.6f}")

Accuracy: 0.954545
F1-score: 0.761905
Recall: 0.800000
Precision: 0.727273


### 3.2 'Обобщение' логистической регрессии

Напишите многоклассовый классификатор. Обучите его на наборе данных ниже.

In [1487]:
x_train, y_train, x_test, y_test = load_data('dataset3')

<b>Ансамбль логистических регрессий.</b> Сложность: супергерой.

In [1488]:
"""
Напишите класс, что инкапсулирует в себе `C` логистических регрессий, 
где `C` - количество классов. i-ая логистическая регрессия производит 
бинарную классификацию вида: все остальные классы и i-ый класс.
"""

class MulticlassLogisticRegression:
    def __init__(self, n_classes, dim=2):

        self.n_classes = n_classes
        self.dim = dim
        self.models = [LogisticRegression(dim) for _ in range(n_classes)]
    
    def fit(self, x, y, iters=1000, lr=0.01):

        for i in range(self.n_classes):
            y_binary = (y == i).astype(int)
            self.models[i].fit(x, y_binary, iters=iters, lr=lr)
    
    def predict(self, x):
         # x - numpy массив размерности [N, dim]
        # Возвращается массив целых чисел размерности [N],
        # где i-ый элемент обозначает номер класса для 
        # i-го экземпляра данных в `x`.
        probabilities = np.zeros((x.shape[0], self.n_classes))
        for i in range(self.n_classes):
            probabilities[:, i] = self.models[i].predict(x, probs=True).flatten()
        
        return np.argmax(probabilities, axis=1)

In [1489]:
#Cмотрим сколько классов
y_test = y_test.astype(int)
        
class_counts = np.bincount(y_test)
for class_label, count in enumerate(class_counts):
    print(f"Класс {class_label}: {count}")

print(f"Количество классов: {len(np.unique(y_test))}")

Класс 0: 48
Класс 1: 39
Класс 2: 52
Класс 3: 49
Класс 4: 39
Класс 5: 50
Класс 6: 42
Класс 7: 43
Класс 8: 37
Класс 9: 51
Количество классов: 10


In [1490]:
# Создайте и обучите написанный классификатор. Оцените точность модели.
model = MulticlassLogisticRegression(n_classes=10, dim=64)

model.fit(x_train, y_train)

In [1491]:
predictions= model.predict(x_test)

accuracy = accuracy_score(y_test, predictions)
print(f"Accuracy: {accuracy:.6f}")

f1 = f1_score(y_test, predictions, average='weighted')
print(f"F1-score: {f1:.6f}")
        
recall = recall_score(y_test, predictions, average='weighted')
print(f"Recall: {recall:.6f}")
        
precision = precision_score(y_test, predictions, zero_division=0, average='weighted')
print(f"Precision: {precision:.6f}")

#Точность вроде бы неплохая

Accuracy: 0.935556
F1-score: 0.932422
Recall: 0.935556
Precision: 0.937119


<b>Softmax классификатор.</b> Сложность: математический гений.

In [1492]:
"""
Напишите класс классификатора, основанного на функции Softmax.
Алгоритм работы данного классификатора:
x - вектор (экземпляр данных) размерности dim.
W - матрица весов размерности [dim, n_classes].

Ответ классификатора формируется как:
logits = x * W - матричное умножение
p = softmax(logits)
class_id = argmax(p)

Для данного классификатора требуется модифицировать алгоритм обучения в методе fit.

Вспомогательные ресурсы:
https://en.wikipedia.org/wiki/Softmax_function
https://eli.thegreenplace.net/2016/the-softmax-function-and-its-derivative/
"""

class SoftmaxClassificator:
    def __init__(self, n_classes, dim):
        pass
    
    def predict(self, x):
        # x - numpy массив размерности [N, dim]
        # Возвращается массив целых чисел размерности [N],
        # где i-ый элемент обозначает номер класса для 
        # i-го экземпляра данных в `x`.
        pass
    
    def fit(self, x, y):
        pass

In [1493]:
# Создайте и обучите написанный классификатор. Оцените точность модели, посчитайте матрицу ошибок (выведите её с помощью matplotlib).


In [1494]:
# Создайте и обучите написанный классификатор на наборе данных из задания 1 (опционально). 
# Оцените точность модели, посчитайте матрицу ошибок (выведите её с помощью matplotlib).
